In [1]:
import sys
repo_dir = ".."
if repo_dir not in sys.path: sys.path.append(repo_dir)
!{sys.executable} -m pip install -e ..[dev] >/dev/null

Some configuration for an AWESOME demo!  
(you can ignore this or play with it)  

> **pro tip:**
> add a bunch of imports and configs to your [`PYTHONSTARTUP`](https://docs.python.org/3/using/cmdline.html#envvar-PYTHONSTARTUP) file 
> (like `from dimipy.units import *`).
> You have now an enhanced calculator!

In [2]:
import dimipy
fractions:bool = True
dimipy.params.update(
    str_formatter=dimipy.formatters.PrettyFormatter(amount_spacing=" ",unit_color=True,fractions=fractions),
    repr_formatter=dimipy.formatters.PrettyFormatter(amount_spacing=" ",unit_color=True,fractions=fractions),
    display_formatter=dimipy.formatters.LatexFormatter(amount_spacing=r"\ ",unit_color=True,fractions=fractions),
    #_unitary_scale=1., _unitary_amount=1.,
)

## Self contained examples

In [3]:
from dimipy.units import * # see source file for exhaustive list of standard units
from dimipy.constants import c,G

u_age       =      13.799e9*y_                 # y_ is defined as (s_*31556925)
u_radius    =m_+   c * u_age                   # checks that result is a length, unit will still be (m_*y_/s_) = (m_*31556925)
u_crit_mass =kg_+  0.5 * c**2 / G * u_radius   # checks that result is a mass
u_mass      =      1e53*kg_                    # https://en.wikipedia.org/wiki/Universe

print( u_crit_mass       ) # unit is kg_*31556925 = Unit[31556925 * M]
print( u_crit_mass  +kg_ ) # print it in standard units

print( u_mass / u_crit_mass      ) # unit is s_/y_ = Unit[3.1688765e-08 * 1]
print( u_mass / u_crit_mass  +0. ) # print it as a Unit.SCALAR

2.7853123608316444e+45 (31556925M)
87895893272337138379613529519400592167777687618715648 (1M)
35902615.95297045 (3.17⋅10⁻⁸)
1.137709582063856


### Temperature

In [4]:
from dimipy.units.si import K_, zero_celsius
from dimipy.units.imperial import Ra_, zero_farenheit
T_absolute = 300.*K_

T_celsius = 27.*K_  # use Kelvin instead of Celsius
T_celsius = T_absolute -zero_celsius
T_absolute = T_celsius +zero_celsius

T_farenheit = 80.*Ra_  # use Rankine instead of Farenheit
T_farenheit = T_absolute -zero_farenheit
T_farenheit = T_celsius +zero_celsius-zero_farenheit
T_absolute = T_farenheit +zero_farenheit+K_  # if Kelvin desired

## Create custom dimensions, units and quantities

In [5]:
from dimipy import Dimension, Unit, Quantity

### Dimensions

`Dimension(**symbols)` is effectively a subtype of `dict`.
Any symbol amount can be retreived with `dim[symbol]`, nonregistred symbols will return `0`.

`Dimension` objects are immutable once created.

In [6]:
length = Dimension(L=1)
speed  = Dimension(L=1, T=-1)
acceleration = Dimension( {'L':1, 'T':-2} )

### Units

`Unit(scale:Numeric, dim:Dimension)` also accepts any number of `*args` (`Dimension`,`Unit`,`Quantity` or unitless objects) as well as dimension `**symbols`, the resulting unit will be the product of all of its arguments.
If `scale` is not specified, it is set to `dimipy.params['_base_scale']` (equal to `1`).

`scale` is an arbitrary non-zero numeric value; only relative scale between different units of the same dimension matters.
The convention is that reference units (e.g. SI units) have scale `1`.

`Unit` objects are immutable once created.

In [7]:
m_    = Unit(scale=1, dim=length)       # a unit is composed of a dimension and a scale
km_   = Unit(scale=1000, dim=length)    # scale is relative to the SI-unit
kn_   = Unit(0.514444,speed)            # the knot definition
kg_   = Unit(1,M=1)     # the dimension parameters can be passed as kwargs
s_    = Unit(T=1)       # a unit with scale 1 (default)
min_  = Unit(60,T=1)    # a unit with scale 60
h_    = s_ * 3600       # a unit with scale 3600 (s_ MUST be on left)
d_    = h_ * 24         # a unit with scale 3600 * 24 = 86400
km_h_ = km_ / h_        # equivalent to Unit(1000/3600, speed)
N_    = Unit(M=1,L=1,T=-2)
J_    = Unit(M=1,L=2,T=-2) +N_*m_ # checks the compatibility of units (homogeneity)
W_    = Unit(M=1,L=2,T=-3) +J_/s_ # actually, W_ takes the value of (J_/s_), the last term

### Quantities

`Quantity(amount:Numeric, unit:Unit)` also accepts any number of `*args` (`Unit`,`Quantity` or unitless objects, but no `Dimension`), the resulting unit will be the product of all of its arguments.
If `amount` is not specified, it is set to `0`.

Contrary to `Dimension` and `Unit`, `Quantity` objects can be altered once created.

In [8]:
g = Quantity(amount=9.81, unit=m_*s_**-2)  # a quantity is composed of a unit and an amount
c = 299792458 * (m_/s_)             # ((m_/s_) MUST be on right)
sun_earth = (8*min_ + 20*s_) / c    # operation between quantities (result is a distance)
au_ = Unit(sun_earth)               # create a unit from a quantity

In [9]:
time = 9*h_ + 80.1*min_     # first convert 9*h_ in min_, then add
time.convert(d_)            # convert time in place in minutes
time.convert()              # convert time in place in SI, i.e. in s_
time2 = time +d_            # Quantity + Unit -> convert the quantity (time2 in d_)
time3 = d_+ time            # Unit + Quantity -> only checks the compatibility (time3 still in s_)
time3 += d_                 # same as time3.convert(d_)
b1 = (time2 == time3)       # True
b2 = ( 12*d_ < 1e6*s_ )     # False
print( c +km_h_ )           # print the speed of light in kilometers per hour
print( f"{c /(km_/h_)+0.} km/h" ) # print the converted amount; adding zero converts to Unit.SCALAR
print( f"{time //h_:n}:{time%h_ //min_:02n}:{time%min_ //s_:02n}" ) # prints '10:20:06'

5396264244/5 (5/18LT⁻¹)
1079252848.8 km/h
10:20:06


## Formatters
See  [`dimipy.formatters`](formatters.py) for available formatters.

In [10]:
import dimipy
dimipy.params.update(
    str_formatter=dimipy.formatters.PrettyFormatter(amount_spacing=" ",unit_color=True,fractions=False),
    repr_formatter=dimipy.formatters.PrettyFormatter(amount_spacing=" ",unit_color=True,fractions=False),
    display_formatter=dimipy.formatters.LatexFormatter(amount_spacing=r"\ ",unit_color=True,fractions=False),
)


In [11]:
from dimipy.units import *
from dimipy.formatters import *
quantity = 1.21*GW_
print(Formatter().format(quantity))
print(PrettyFormatter().format(quantity))
print(PrettyFormatter(amount_spacing=" ",dim_spacing="⋅").format(quantity))
print(LegacyFormatter(unit_color=False).format(quantity))
print(CodeFormatter().format(quantity))
print(CodeFormatter(explicit_type=True).format(quantity))
print(LatexFormatter(amount_spacing=r"\ ",dim_spacing=r"\cdot").format(quantity))

from IPython.display import display
display(LatexFormatter().Display(quantity))
display(LatexFormatter(amount_spacing=r"\ ",dim_spacing=r"\cdot").Display(quantity))

1.21 (10^9 M L^2 T^-3)
1.21(10⁹ML²T⁻³)
1.21 (10⁹⋅M⋅L²⋅T⁻³)
Quantity[1.21 * (10⁹ * M L² T⁻³)]
1.21 * Unit(Fraction(1000000000, 1), M=1, L=2, T=-3)
Quantity(1.21, Unit(Fraction(1000000000, 1), M=1, L=2, T=-3))
$1.21\ {\color{cyan}\left({10}^{9}\cdot\mathsf{M}\cdot\mathsf{L}^{2}\cdot\mathsf{T}^{-3}\right)}$


$1.21{\color{cyan}\left({10}^{9}\mathsf{M}\mathsf{L}^{2}\mathsf{T}^{-3}\right)}$

$1.21\ {\color{cyan}\left({10}^{9}\cdot\mathsf{M}\cdot\mathsf{L}^{2}\cdot\mathsf{T}^{-3}\right)}$

In [12]:
from dimipy.units import *
quantity = 1.21*GW_
values = [quantity, quantity.unit, quantity.unit.dim]

from dimipy.formatters import Formatter
import dimipy.formatters
formatters = []
for name,var in dimipy.formatters.__dict__.items():
    if not isinstance(var,type): continue
    if not issubclass(var,Formatter): continue
    formatter = var()
    formatters.append(formatter)
formatters.extend([
    dimipy.formatters.CodeFormatter(explicit_type=True),
    dimipy.formatters.CodeFormatter(arg_spacing=""),
])

for formatter in formatters:
    # print(f"{formatter!r:>15}: "+" ".join(
    #     f"{formatter.format(value):90}"
    #     for value in values
    # ))

    for value in values:
        print(f"{repr(formatter)+',':35} {value.__class__.__qualname__+':':10} {formatter.format(value)}")
    print()

Formatter(),                        Quantity:  1.21 (10^9 M L^2 T^-3)
Formatter(),                        Unit:      (10^9 M L^2 T^-3)
Formatter(),                        Dimension: [M L^2 T^-3]

PrettyFormatter(),                  Quantity:  1.21(10⁹ML²T⁻³)
PrettyFormatter(),                  Unit:      (10⁹ML²T⁻³)
PrettyFormatter(),                  Dimension: [ML²T⁻³]

LegacyFormatter(),                  Quantity:  Quantity[1.21 * (10⁹ * M L² T⁻³)]
LegacyFormatter(),                  Unit:      Unit[10⁹ * M L² T⁻³]
LegacyFormatter(),                  Dimension: Dimension[M L² T⁻³]

LatexFormatter(),                   Quantity:  $1.21{\color{cyan}\left({10}^{9}\mathsf{M}\mathsf{L}^{2}\mathsf{T}^{-3}\right)}$
LatexFormatter(),                   Unit:      ${\color{cyan}\left({10}^{9}\mathsf{M}\mathsf{L}^{2}\mathsf{T}^{-3}\right)}$
LatexFormatter(),                   Dimension: ${\color{cyan}\left[\mathsf{M}\mathsf{L}^{2}\mathsf{T}^{-3}\right]}$

CodeFormatter(),                    Qua

## Tests

In [13]:
from dimipy import Unit, formatters
from fractions import Fraction
scales = [
    1e300,
    1e-300,
    1.23456789e300,
    1.23456789e-300,
    10**-60,
    10**20,
    10**-2,
    2**6,
    5*10**5,
    5.1*10**7,
    2/3,
    2/3000,
    Fraction("2/3000000"),
]
for e in range(18): scales.append(123456789*10**-e)
for e in range(-3,3+1): scales.append(10**e)
for e in range(-4,7+1): scales.append(2**e)
formatter = formatters.PrettyFormatter(
    fractions=True,  # fraction display with inference
    # fractions=None,  # fraction display without inference
    # fractions=False, # fraction rounded
)
for scale in scales:
    print(f"{formatter.format(Unit(scale)):22} from {scale!r}")

(10³⁰⁰)       from 1e+300
(10⁻³⁰⁰)      from 1e-300
(1.23⋅10³⁰⁰)  from 1.23456789e+300
(1.23⋅10⁻³⁰⁰) from 1.23456789e-300
(10⁻⁶⁰)       from 1e-60
(10²⁰)        from 100000000000000000000
(10⁻²)        from 0.01
(64)          from 64
(5⋅10⁵)       from 500000
(5.1⋅10⁷)     from 51000000.0
(2/3)         from 0.6666666666666666
(1/1500)      from 0.0006666666666666666
(1/1500000)   from Fraction(1, 1500000)
(1.23⋅10⁸)    from 123456789
(12345679)    from 12345678.9
(1234568)     from 1234567.8900000001
(123457)      from 123456.789
(12346)       from 12345.6789
(1235)        from 1234.56789
(123.5)       from 123.456789
(12.35)       from 12.3456789
(1.235)       from 1.2345678900000001
(0.1235)      from 0.12345678900000001
(0.01235)     from 0.0123456789
(0.001235)    from 0.00123456789
(0.0001235)   from 0.000123456789
(1.23⋅10⁻⁵)   from 1.23456789e-05
(1.23⋅10⁻⁶)   from 1.23456789e-06
(1.23⋅10⁻⁷)   from 1.2345678900000002e-07
(1.23⋅10⁻⁸)   from 1.23456789e-08
(1.23⋅10⁻⁹)   from 1.234